<a href="https://colab.research.google.com/github/pawarad/ADS/blob/master/Fast_parapred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
!pip install Biopython

In [116]:
from Bio.PDB import *
from Bio.PDB.Model import Model
from Bio.PDB.Structure import Structure
import numpy as np
import Bio.PDB
import pickle
import pandas as pd

In [117]:
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)

In [118]:
cdr_loop = pd.read_pickle("/content/drive/My Drive/Peritia_Fast-Parapred/CDRs_seqno_pdb.pkl")


In [119]:
def residue_in_contact_with(res, ab_search, dist):
    return any(len(ab_search.search(a.coord, dist)) > 0
               for a in res.get_unpacked_list())

In [120]:
def aaMap(_aa):
    aa_df = pd.DataFrame([['Alanine','Ala','A'],
                        ['Arginine','Arg','R'],
                        ['Asparagine','Asn','N'],
                        ['Aspartate','Asp','D'],
                        ['Cysteine','Cys','C'],
                        ['Glutamate','Glu','E'],
                        ['Glutamine','Gln','Q'],
                        ['Glycine','Gly','G'],
                        ['Histidine','His','H'],
                        ['Histidine_D','HID','H'],
                        ['Histidine_E','HIE','H'],
                        ['Histidine_P','HIP','H'],
                        ['Isoleucine','Ile','I'],
                        ['Leucine','Leu','L'],
                        ['Lysine','Lys','K'],
                        ['Methionine','Met','M'],
                        ['Phenylalanine','Phe','F'],
                        ['Proline','Pro','P'],
                        ['Serine','Ser','S'],
                        ['Threonine','Thr','T'],
                        ['Tryptophan','Trp','W'],
                        ['Tyrosine','Tyr','Y'],
                        ['Valine','Val','V']],columns=['Full','3','1'])
    aa_df['3'] = aa_df['3'].str.upper()
    _aa = _aa.upper()
    if len(_aa)==1:
        toret = aa_df.loc[aa_df['1']==_aa,'3'].values
        if len(toret)>0:
            toret = toret[0]
        else:
            return None
    else:
        toret = aa_df.loc[aa_df['3']==_aa,'1'].values
        if len(toret)>0:
            toret = toret[0]
        else:
            return None
    return toret

def maxi_len(all_variables):
    max_length = 0
    for i in range(len(all_variables)):
        length = len(all_variables[i])
        max_length = max(max_length,length)
    #     max_length = length
    return max_length


def ext_epitope(H_chain,L_chain,A_chain,pdb):
    p = PDBParser()
    structure = p.get_structure('X', "/content/drive/My Drive/Peritia_Fast-Parapred/cleaned_pdb_shweta/"+pdb+'.pdb')
    print(pdb)
    ## construct a Model with residues of Heavy and Light chain of the pdb
    model = structure[0]
    model1 = Model(0)
    model1.add(model[H_chain]) ## heavy chain
    model1.add(model[L_chain]) ## Light chain
    
#     print(list(model1))

    ## construct another Model with residues of Antigen chain of the pdb
    model2 = Model(1)
    for i in A_chain:
        model2.add(model[i])
#     print(list(model2))
    
    ## Search for nearest atoms within the Antibody(Heavy/Light chain)
    ab_search = NeighborSearch(Selection.unfold_entities(model1, 'A'))

    epitope = []
    for r in model2:
        for j in r:
            if residue_in_contact_with(j, ab_search,5) == True:
                epitope.append(j)

    epi_residues =[]
    epi_res_number = []
    for i in epitope:
        tags = i.id
    #     print(tags)
        if tags[0] == " ":
            res_name = i.get_resname() 
            res_abbre = aaMap(res_name)
            epi_residues.append(res_abbre)
            epi_res_number.append(str(tags[1])+str(tags[2]))
#     print("epi_res",epi_residues) 
#     print("epi_res_number",epi_res_number )

    # print("tags",sorted(epitope))

    epi_search = NeighborSearch(Selection.unfold_entities(epitope, 'A'))

    ext_epi =[]
    for r in model2:
        for j in r:
            if residue_in_contact_with(j, epi_search,10) == True:
                ext_epi.append(j)

    ext_epi_residues = []
    ext_epi_res_number = []
    for i in ext_epi:
        tags = i.id
    #     print("tags",sorted(tags[1]))
        if tags[0] == " ":
            res_name = i.get_resname() 
            res_abbre = aaMap(res_name)
            ext_epi_residues.append(res_abbre)
    #         print(i,res_name)
            ext_epi_res_number.append(str(tags[1])+str(tags[2]))
#     print("ext_epi_res_number",ext_epi_res_number)
#     print("ext_epi_res",ext_epi_residues)


    return epi_residues, epi_res_number, ext_epi_res_number, ext_epi_residues



def ext_epitope_sorted(H_chain,L_chain,A_chain,pdb):
    p = PDBParser()
    structure = p.get_structure('X', "/content/drive/My Drive/Peritia_Fast-Parapred/cleaned_pdb_shweta/"+pdb+'.pdb')
    print(pdb)
    ## construct a Model with residues of Heavy and Light chain of the pdb
    model = structure[0]
    model1 = Model(0)
    model1.add(model[H_chain]) ## heavy chain
    model1.add(model[L_chain]) ## Light chain
    
#     print(list(model1))

    ## construct another Model with residues of Antigen chain of the pdb
    model2 = Model(1)
    for i in A_chain:
        model2.add(model[i])
#     print(list(model2))
    
    ## Search for nearest atoms within the Antibody(Heavy/Light chain)
    ab_search = NeighborSearch(Selection.unfold_entities(model1, 'A'))

    epitope = []
    for r in model2:
        for j in r:
            if residue_in_contact_with(j, ab_search,5) == True:
                epitope.append(j)

    epi_residues =[]
    epi_res_number = []
    for i in sorted(epitope):
        tags = i.id
    #     print(tags)
        if tags[0] == " ":
            res_name = i.get_resname() 
            res_abbre = aaMap(res_name)
            epi_residues.append(res_abbre)
            epi_res_number.append(str(tags[1])+str(tags[2]))
#     print("epi_res",epi_residues) 
#     print("epi_res_number",epi_res_number )

    # print("tags",sorted(epitope))

    epi_search = NeighborSearch(Selection.unfold_entities(epitope, 'A'))

    ext_epi =[]
    for r in model2:
        for j in r:
            if residue_in_contact_with(j, epi_search,10) == True:
                ext_epi.append(j)

    ext_epi_residues = []
    ext_epi_res_number = []
    for i in sorted(ext_epi):
        tags = i.id
    #     print("tags",sorted(tags[1]))
        if tags[0] == " ":
            res_name = i.get_resname() 
            res_abbre = aaMap(res_name)
            ext_epi_residues.append(res_abbre)
    #         print(i,res_name)
            ext_epi_res_number.append(str(tags[1])+str(tags[2]))
#     print("ext_epi_res_number",ext_epi_res_number)
#     print("ext_epi_res",ext_epi_residues)


    return epi_residues, epi_res_number, ext_epi_res_number, ext_epi_residues


aa_s = "CSTPAGNDEQHRKMILVFYWX"
def one_to_number(res_str):
    return [aa_s.index(r) for r in res_str]

def aa_features():
    # Meiler's features
    prop1 = [[1.77, 0.13, 2.43,  1.54,  6.35, 0.17, 0.41],
             [1.31, 0.06, 1.60, -0.04,  5.70, 0.20, 0.28],
             [3.03, 0.11, 2.60,  0.26,  5.60, 0.21, 0.36],
             [2.67, 0.00, 2.72,  0.72,  6.80, 0.13, 0.34],
             [1.28, 0.05, 1.00,  0.31,  6.11, 0.42, 0.23],
             [0.00, 0.00, 0.00,  0.00,  6.07, 0.13, 0.15],
             [1.60, 0.13, 2.95, -0.60,  6.52, 0.21, 0.22],
             [1.60, 0.11, 2.78, -0.77,  2.95, 0.25, 0.20],
             [1.56, 0.15, 3.78, -0.64,  3.09, 0.42, 0.21],
             [1.56, 0.18, 3.95, -0.22,  5.65, 0.36, 0.25],
             [2.99, 0.23, 4.66,  0.13,  7.69, 0.27, 0.30],
             [2.34, 0.29, 6.13, -1.01, 10.74, 0.36, 0.25],
             [1.89, 0.22, 4.77, -0.99,  9.99, 0.32, 0.27],
             [2.35, 0.22, 4.43,  1.23,  5.71, 0.38, 0.32],
             [4.19, 0.19, 4.00,  1.80,  6.04, 0.30, 0.45],
             [2.59, 0.19, 4.00,  1.70,  6.04, 0.39, 0.31],
             [3.67, 0.14, 3.00,  1.22,  6.02, 0.27, 0.49],
             [2.94, 0.29, 5.89,  1.79,  5.67, 0.30, 0.38],
             [2.94, 0.30, 6.47,  0.96,  5.66, 0.25, 0.41],
             [3.21, 0.41, 8.08,  2.25,  5.94, 0.32, 0.42],
             [0.00, 0.00, 0.00,  0.00,  0.00, 0.00, 0.00]]
    return np.array(prop1)

NUM_FEATURES = len(aa_s) + 7

def to_categorical(y, num_classes):
    """ Converts a class vector to binary class matrix. """
    new_y = torch.LongTensor(y)
    n = new_y.size()[0]
    categorical = torch.zeros(n, num_classes)
    arangedTensor = torch.arange(0, n)
    intaranged = arangedTensor.long()
    categorical[intaranged, new_y] = 1
    return categorical

def seq_to_one_hot(res_seq_one):
#     from keras.utils.np_utils import to_categorical
    ints = one_to_number(res_seq_one)
    new_ints = torch.LongTensor(ints)
    feats = torch.Tensor(aa_features()[new_ints])
    onehot = to_categorical(ints, num_classes=len(aa_s))
    return torch.cat((onehot, feats), axis=1)

def cdrseq_to_one_hot(res_seq_one,i):
#     from keras.utils.np_utils import to_categorical
    ints = one_to_number(res_seq_one)
    new_ints = torch.LongTensor(ints)
    feats = torch.Tensor(aa_features()[ints])
    onehot = to_categorical(ints, num_classes=len(aa_s))
    if i%6 == 0:
        ext_onehot = [1, 0, 0, 0, 0, 0]
    if i%6 == 1:
        ext_onehot = [0, 1, 0, 0, 0, 0]
    if i%6 == 2:
        ext_onehot = [0, 0, 1, 0, 0, 0]
    if i%6 == 3:
        ext_onehot = [0, 0, 0, 1, 0, 0]
    if i%6 == 4:
        ext_onehot = [0, 0, 0, 0, 1, 0]
    if i%6 == 5:
        ext_onehot = [0, 0, 0, 0, 0, 1]
    
    chain_encoding = torch.Tensor(ext_onehot)
    chain_encoding = chain_encoding.expand(onehot.shape[0], 6)
    concatenated = torch.cat((onehot, feats,chain_encoding), 1)

    return concatenated



Epitope Cross Model

In [121]:
from __future__ import print_function
from sklearn.model_selection import KFold

import numpy as np
# np.set_printoptions(threshold=np.nan)
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch import index_select

In [122]:
MAX_CDR_LENGTH = 38

MAX_EXT_AG_LENGTH = 288

use_cuda = torch.cuda.is_available()

epochs = 16

batch_size = 3

class EpitopeX(nn.Module):
    def __init__(self):
        """
        Declares the building blocks of the neural network.
        """
        super(EpitopeX, self).__init__()
        

        #self.conv1 = nn.Conv1d(input_layer = NUM_FEATURES, output_channel/filter = 64, filter_size = 3, padding=1)
        self.conv1 = nn.Conv1d(NUM_FEATURES, 64, 3, padding=1)  # antibody first a trous convolutional layer

        self.agconv1 = nn.Conv1d(AG_NUM_FEATURES, 64, 3, padding=1)   # antigen first a trous convolutional layer

        self.conv2 = nn.Conv1d(64, 128, 3, padding=2, dilation=2)  #antibody second a trous convolutional layer

        self.agconv2 = nn.Conv1d(64, 128, 3, padding=2, dilation=2)  #antigen second a trous convolutional layer

        self.conv3 = nn.Conv1d(128, 256, 3, padding=4, dilation=4)  #antibody third a trous convolutional layer

        self.agconv3 = nn.Conv1d(128, 256, 3, padding=4, dilation=4)

        #self.agconv4 = nn.Conv1d(256, 32, 1)


        self.bn1 = nn.BatchNorm1d(64)  # batch normalisation after the first convolutional layer for antibody
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)
        self.bn4 = nn.BatchNorm1d(512)

        self.agbn1 = nn.BatchNorm1d(64)  # batch normalisation after the first convolutional layer for antigen
        self.agbn2 = nn.BatchNorm1d(128)
        self.agbn3 = nn.BatchNorm1d(256)

        self.elu = nn.ReLU()            # activation function
        self.dropout = nn.Dropout(0.15)
        self.dropout2 = nn.Dropout(0.5)
        self.fc = nn.Linear(512, 1, 1)  # dense prediction layer
        self.softmax = nn.Softmax(dim=-1)
        self.lrelu = nn.LeakyReLU(0.2)

        self.aconv1 = nn.Conv1d(256, 1, 1)  # attentional mechanism
        self.aconv2 = nn.Conv1d(32, 1, 1)


        self.maxpool1 = nn.MaxPool1d(2)
        self.maxpool2 = nn.MaxPool1d(4)

        for m in self.modules():
            self.weights_init(m)

    def weights_init(self, m):
        if isinstance(m, nn.Conv1d):
            torch.nn.init.xavier_uniform(m.weight.data)
            m.bias.data.fill_(0.0)
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform(m.weight.data)
            m.bias.data.fill_(0.0)

    def forward(self, ag_input, ag_unpacked_masks, ab_input, ab_unpacked_masks):
        """
        Forward propagation step
        :param ab_input: antibody amino acid sequences
        :param ab_unpacked_masks: antibody amino acid masks
        :param ag_input: antigen amino acid sequences
        :param ag_unpacked_masks: antigen amino acid masks
        :param dist: maskin
        :return: antibody binding probabilities, attentional coefficients
        """
        x=ab_input
        agx = ag_input

        ab_unpacked_masks = torch.transpose(ab_unpacked_masks, 1, 2)
        ag_unpacked_masks = torch.transpose(ag_unpacked_masks, 1, 2)

        x = torch.transpose(x, 1, 2)
        agx = torch.transpose(agx, 1, 2)

        x = self.conv1(x) 
        x = torch.mul(x, ab_unpacked_masks)
        x = self.bn1(x)
        x = self.elu(x)
        x = torch.mul(x, ab_unpacked_masks)
        x = self.dropout(x)

        agx = self.agconv1(agx)
        agx = torch.mul(agx, ag_unpacked_masks)
        agx = self.agbn1(agx)
        agx = self.elu(agx)
        agx = torch.mul(agx, ag_unpacked_masks)
        agx = self.dropout(agx)

        x = self.conv2(x)
        x = torch.mul(x, ab_unpacked_masks)
        x = self.bn2(x)
        x = self.elu(x)
        x = torch.mul(x, ab_unpacked_masks)
        x = self.dropout(x)

        #agx = self.maxpool1(agx)
        #ag_unpacked_masks = self.maxpool1(ag_unpacked_masks)

        agx = self.conv2(agx)
        agx = torch.mul(agx, ag_unpacked_masks)
        agx = self.agbn2(agx)
        agx = self.elu(agx)
        agx = torch.mul(agx, ag_unpacked_masks)
        agx = self.dropout(agx)


        x = self.conv3(x)
        x = torch.mul(x, ab_unpacked_masks)
        x = self.bn3(x)
        x = self.elu(x)
        x = torch.mul(x, ab_unpacked_masks)
        x = self.dropout(x)

        #agx = self.maxpool1(agx)
        #ag_unpacked_masks = self.maxpool1(ag_unpacked_masks)

        agx = self.conv3(agx)
        agx = torch.mul(agx, ag_unpacked_masks)
        agx = self.agbn3(agx)
        agx = self.elu(agx)
        agx = torch.mul(agx, ag_unpacked_masks)
        agx = self.dropout(agx)

        # MaxPool
        #agx = self.maxpool1(agx)
        #ag_unpacked_masks = self.maxpool1(ag_unpacked_masks)

        old = x

        oldag = agx

        heads_no = 1


        for i in range(heads_no):
            #agconvi = nn.Conv1d(256, 128, 1)
            aconvi1 = nn.Conv1d(256, 1, 1)
            aconvi2 = nn.Conv1d(256, 1, 1)
            if use_cuda:
                aconvi1.cuda()
                aconvi2.cuda()
                #agconvi.cuda()
            #agx = agconvi(oldag)
            w_1 = aconvi1(x)
            w_2 = aconvi2(agx)
            #w = self.lrelu(w_1 + torch.transpose(w_2, 1, 2))

            # print("w_1.shape", w_1.shape)
            # print("w_2.shape", w_2.shape)

            w= self.lrelu(torch.bmm(torch.transpose(w_1, 1, 2), w_2))

            w = self.softmax(w)
            w = self.dropout(w)
            temp_loop_x = torch.bmm(torch.transpose(w,1,2), torch.transpose(x, 1, 2))
            if i==0:
                loop_x = temp_loop_x
            else:
                loop_x = torch.cat((loop_x, temp_loop_x), dim=2)

        agx = torch.transpose(loop_x, 1, 2)
        #x = x + old
        agx = torch.cat((agx, oldag), dim=1)
        agx = torch.mul(agx, ag_unpacked_masks)

        agx = self.bn4(agx)
        agx = self.elu(agx)
        agx = torch.mul(agx, ag_unpacked_masks)
        agx = torch.transpose(agx, 1, 2)

        agx = self.dropout2(agx)

        agx = self.fc(agx)

        #print("x after fc", x, file=track_f)

        return agx, w

In [123]:
from __future__ import print_function

import numpy as np
# np.set_printoptions(threshold=np.nan)
from torch.autograd import Variable
import torch
import torch.nn as nn
torch.set_printoptions(threshold=50000)
import torch.optim as optim
from torch import squeeze
from torch.optim import lr_scheduler
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from torch import index_select
from sklearn.metrics import confusion_matrix, roc_auc_score, matthews_corrcoef

**Evaluation of Model**

In [124]:
def get_variables(dist_matx_path_heavy,dist_matx_path_light,cdr_loop,cut_off):
    """
    For every CDR get the corresponding seq and check if dist is less than cutoff
    and substitute 1 orelse 0 
    
    E.g. - CDR - TCRASGNIHNYLAWY
           Seq.no - ['22','23','24','25','26','27','28','29','30','31','32','33','34','35','36']
           
           Output - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
    """
    pdb_name = cdr_loop['PDB_Name'].to_numpy()
    seq_no = cdr_loop['seq_no'].to_numpy()
    chain_type = cdr_loop['Type'].to_numpy()

    all_var = []
    for i in range(len(pdb_name)):
        if chain_type[i] == 'H1' or chain_type[i] == 'H2' or chain_type[i] == 'H3':
            dist = pd.read_pickle(dist_matx_path_heavy+pdb_name[i]+"heavy"+".pkl")
            var = []
            for j in range(len(seq_no[i])):
                first_index = dist.index.get_level_values(0)[0] ##get the first index for slicing
                row_slc = dist.loc[(first_index, seq_no[i][j])] ##slice multi index for specific row

                if np.any(row_slc < cut_off) == True:
                    var.append(1)
                else:
                    var.append(0)
            all_var.append(var)

        elif chain_type[i] == 'L1' or chain_type[i] == 'L2' or chain_type[i] == 'L3':
            dist = pd.read_pickle(dist_matx_path_light+pdb_name[i]+"light"+".pkl")
            var = []
            for j in range(len(seq_no[i])):
                first_index = dist.index.get_level_values(0)[0] ##get the first index for slicing
                row_slc = dist.loc[(first_index, seq_no[i][j])] ##slice multi index for specific row

                if np.any(row_slc < cut_off) == True:
                    var.append(1)
                else:
                    var.append(0)
            all_var.append(var)
    
    return all_var

In [125]:
def permute_training_cross_data(cdrs, cdr_masks, cdr_lengths, ag, ag_masks, ag_lengths,ag_lbls):
    index = torch.randperm(cdrs.shape[0])
    if use_cuda:
        index = index.cuda()

    cdrs = torch.index_select(cdrs, 0, index)
    cdr_masks = torch.index_select(cdr_masks, 0, index)
#     cdr_lbls = torch.index_select(cdr_lbls, 0, index)
    cdr_lengths = [cdr_lengths[i] for i in index]

    ag = torch.index_select(ag, 0, index)
    ag_masks = torch.index_select(ag_masks, 0, index)
#     print("permute",type(ag_masks))
    ag_lbls = torch.index_select(ag_lbls, 0, index)
    ag_lengths = [ag_lengths[i] for i in index]

    return cdrs, cdr_masks, cdr_lengths, ag, ag_masks, ag_lengths,  ag_lbls

def flatten_with_lengths(matrix, lengths):
    seqs = []
    for i, example in enumerate(matrix):
        seq = example[:lengths[i]]
        seqs.append(seq)
    return np.concatenate(seqs)

def sort_cross_batch(cdrs, cdr_masks, ag, ag_masks, ag_lbls, ag_lengths):
    order = np.argsort(ag_lengths)
    order = order.tolist()
    order.reverse()

    ag_lengths.sort(reverse=True)

    index = Variable(torch.LongTensor(order))
    if use_cuda:
        index = index.cuda()

    cdrs = torch.index_select(cdrs, 0, index)
    cdr_masks = torch.index_select(cdr_masks, 0, index)

    ag = torch.index_select(ag, 0, index)
    ag_masks = torch.index_select(ag_masks, 0, index)
    ag_lbls = torch.index_select(ag_lbls, 0, index)


    return cdrs, cdr_masks, ag, ag_masks, ag_lbls, ag_lengths

In [126]:
def x_epitope_run(epi_train, lbls_train, masks_train, lengths_train, cdrs_train, cdr_masks_train, cdr_lengths_train,
                  weights_template, weights_template_number, epi_test, lbls_test, masks_test, lengths_test,
                  cdrs_test, cdr_masks_test, cdr_lengths_test):

#     print("epitope run", file=print_file)
    print("epitope run")
    model = EpitopeX()

    ignored_params = list(map(id, [model.conv1.weight,
                                   #model.conv2.weight, model.conv3.weight, model.aconv1.weight,
                                   #model.aconv2.weight
                                ]))
    base_params = filter(lambda p: id(p) not in ignored_params,
                         model.parameters())

    optimizer = optim.Adam([
        {'params': base_params},
        {'params': model.conv1.weight, 'weight_decay': 0.01},
        #{'params': model.conv2.weight, 'weight_decay': 0.01},
        #{'params': model.conv3.weight, 'weight_decay': 0.01},
        #{'params': model.aconv1.weight, 'weight_decay': 0.01},
        #{'params': model.aconv2.weight, 'weight_decay': 0.01}
    ], lr=0.01)

    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10], gamma=0.1)

    total_input = epi_train
    total_lbls = lbls_train
    total_masks = masks_train
    total_lengths = lengths_train
#     print(type(total_masks))

    total_cdrs_train = cdrs_train
    total_cdr_masks_train = cdr_masks_train
    total_cdr_lengths_train = cdr_lengths_train

    if use_cuda:
        print("using cuda")
        model.cuda()
        total_input = total_input.cuda()
        total_lbls = total_lbls.cuda()
        total_masks = total_masks.cuda()
        epi_test = epi_test.cuda()
        lbls_test = lbls_test.cuda()
        masks_test = masks_test.cuda()

        total_cdrs_train = total_cdrs_train.cuda()
        total_cdr_masks_train = total_cdr_masks_train.cuda()
        cdrs_test = cdrs_test.cuda()
        cdr_masks_test = cdr_masks_test.cuda()

    for epoch in range(epochs):
        model.train(True)
        scheduler.step()
        epoch_loss = 0

        batches_done=0

        #total_input, total_masks, total_lengths, total_lbls = \
        #    permute_training_data(total_input, total_masks, total_lengths, total_lbls)

        total_cdrs_train, total_cdr_masks_train, total_cdr_lengths_train, total_input, total_masks, total_lengths, total_lbls = \
            permute_training_cross_data(cdrs=total_cdrs_train, cdr_masks=total_cdr_masks_train, cdr_lengths=total_cdr_lengths_train,
                        ag=total_input, ag_masks=total_masks, ag_lengths=total_lengths, ag_lbls=total_lbls)
        
#         print("total_masks_output",type(total_masks))

        for j in range(0, epi_train.shape[0], batch_size):
            batches_done +=1
            interval = [x for x in range(j, min(epi_train.shape[0], j + batch_size))]
            interval = torch.LongTensor(interval)
            if use_cuda:
                interval = interval.cuda()
#             print("total_input",type(total_input))
#             print("total_masks",type(total_masks))

            input = Variable(index_select(total_input, 0, interval), requires_grad=True)
            masks = Variable(index_select(total_masks, 0, interval))
            lengths = total_lengths[j:j + batch_size]
            lbls = Variable(index_select(total_lbls, 0, interval))

            cdrs_train = Variable(index_select(total_cdrs_train, 0, interval))
            cdr_masks_train = Variable(index_select(total_cdr_masks_train, 0, interval))
            cdr_lengths_train = total_cdr_lengths_train[j:j+batch_size]

            cdrs, cdr_masks, input, masks, lbls, lengths = sort_cross_batch(cdrs=cdrs_train, cdr_masks=cdr_masks_train,
                                               ag=input, ag_masks=masks, ag_lbls=lbls,ag_lengths=list(lengths))

            unpacked_masks = masks

            packed_masks = pack_padded_sequence(masks, lengths, batch_first=True)
            masks, _ = pad_packed_sequence(packed_masks, batch_first=True)

            unpacked_lbls = lbls

            packed_lbls = pack_padded_sequence(lbls, lengths, batch_first=True)
            lbls, _ = pad_packed_sequence(packed_lbls, batch_first=True)


            output, attn_coeff = model(input, unpacked_masks, cdrs, cdr_masks)

            loss_weights = (unpacked_lbls * 1.5 + 1) * unpacked_masks
            max_val = (-output).clamp(min=0)
            loss = loss_weights * (output - output * unpacked_lbls + max_val + ((-max_val).exp() + (-output - max_val).exp()).log())
            masks_added = masks.sum()
            loss = loss.sum() / masks_added

            #print("Epoch %d - Batch %d has loss %d " % (epoch, j, loss.data), file=monitoring_file)
            epoch_loss +=loss

            model.zero_grad()

            loss.backward()
            optimizer.step()
        print("Epoch %d - loss is %f : " % (epoch, epoch_loss.data/batches_done))

        model.eval()

        cdrs_test2, cdr_masks_test2, epi_test2, masks_test2, lbls_test2, lengths_test2 = \
            sort_cross_batch(cdrs=cdrs_test, cdr_masks=cdr_masks_test, ag=epi_test, ag_masks=masks_test,
                                         ag_lbls=lbls_test,ag_lengths=list(lengths_test))

        #epi_test2, masks_test2, lengths_test2, lbls_test2 = sort_batch(epi_test, masks_test, list(lengths_test),
        #                                                            lbls_test)

        unpacked_masks_test2 = masks_test2

        probs_test2, attn_coeff_test2 = model(epi_test2, unpacked_masks_test2, cdrs_test2, cdr_masks_test2)

        # K.mean(K.equal(lbls_test, K.round(y_pred)), axis=-1)

        sigmoid = nn.Sigmoid()
        probs_test2 = sigmoid(probs_test2)

        probs_test2 = probs_test2.data.cpu().numpy().astype('float32')
        lbls_test2 = lbls_test2.data.cpu().numpy().astype('int32')

        probs_test2 = flatten_with_lengths(probs_test2, lengths_test2)
        lbls_test2 = flatten_with_lengths(lbls_test2, lengths_test2)

        print("Roc", roc_auc_score(lbls_test2, probs_test2))
        # print("lbls_test2, probs_test2:",(lbls_test2, probs_test2))

    torch.save(model.state_dict(), weights_template.format(weights_template_number))

    print("test")
    model.eval()

    cdrs_test, cdr_masks_test, epi_test, masks_test, lbls_test, lengths_test = \
        sort_cross_batch(cdrs=cdrs_test, cdr_masks=cdr_masks_test, ag=epi_test, ag_masks=masks_test,
                         ag_lbls=lbls_test,ag_lengths=list(lengths_test))

    #epi_test, masks_test, lengths_test, lbls_test = sort_batch(epi_test, masks_test, list(lengths_test), lbls_test)

    unpacked_masks_test = masks_test
    packed_input = pack_padded_sequence(masks_test, list(lengths_test), batch_first=True)
    masks_test, _ = pad_packed_sequence(packed_input, batch_first=True)

    probs_test, attn_coeff_test = model(epi_test, unpacked_masks_test, cdrs_test, cdr_masks_test)

    # K.mean(K.equal(lbls_test, K.round(y_pred)), axis=-1)

    sigmoid = nn.Sigmoid()
    probs_test = sigmoid(probs_test)

    probs_test1 = probs_test.data.cpu().numpy().astype('float32')
    lbls_test1 = lbls_test.data.cpu().numpy().astype('int32')

    probs_test1 = flatten_with_lengths(probs_test1, list(lengths_test))
    lbls_test1 = flatten_with_lengths(lbls_test1, list(lengths_test))

    print("Roc", roc_auc_score(lbls_test1, probs_test1))
    # print("lbls_test1, probs_test1:",(lbls_test1, probs_test1))

    return probs_test, lbls_test, probs_test1, lbls_test1  # get them in kfold, append, concatenate do roc on them

In [127]:
def kfold_cv_eval(ag, lbls, masks, lengths, cdrs, cdr_masks, cdr_lengths, seed,output_file,
                  weights_template):
#     ag, lbls, masks, lengths, cdrs, cdr_masks, cdr_lengths = \
#         dataset["ag"], dataset["ag_lbls"], dataset["ag_masks"], dataset["ag_lengths"], \
#         dataset["cdrs"], dataset["cdr_masks"], dataset["cdr_lengths"]

    print("ag", ag.shape)
    print("lbls", lbls.shape)
    print("masks", masks.shape)
    print("cdrs", cdrs.shape)
    print("cdr masks", cdr_masks.shape)

    kf = KFold(n_splits=NUM_SPLIT, random_state=seed, shuffle=True)

    all_lbls2 = []
    all_probs2 = []
    all_masks = []

    all_probs1 = []
    all_lbls1 = []

    for i, (train_idx, test_idx) in enumerate(kf.split(ag)):
        print("train_idx, test_idx:",(train_idx, test_idx))
        print("Fold: ", i + 1)

        lengths_train = [lengths[i] for i in train_idx]
        lengths_test = [lengths[i] for i in test_idx]

        cdr_lengths_train = [cdr_lengths[i] for i in train_idx]
        cdr_lengths_test = [cdr_lengths[i] for i in test_idx]

        print("len(train_idx)",len(train_idx))

        train_idx = torch.from_numpy(train_idx)
        test_idx = torch.from_numpy(test_idx)

        ag_train = index_select(ag, 0, train_idx)
        lbls_train = index_select(lbls, 0, train_idx)
        masks_train = index_select(masks, 0, train_idx)
        cdrs_train = index_select(cdrs, 0, train_idx)
        cdr_masks_train = index_select(cdr_masks, 0, train_idx)


        ag_test = Variable(index_select(ag, 0, test_idx))
        lbls_test = Variable(index_select(lbls, 0, test_idx))
        masks_test = Variable(index_select(masks, 0, test_idx))
        cdrs_test = Variable(index_select(cdrs, 0, test_idx))
        cdr_masks_test = Variable(index_select(cdr_masks, 0, test_idx))
        
        

        code = 2
        if code == 1:
            probs_test1, lbls_test1, probs_test2, lbls_test2 = \
                epitope_run(ag_train, lbls_train, masks_train, lengths_train, weights_template, i,
                                    ag_test, lbls_test, masks_test, lengths_test)

        if code == 2:
            probs_test1, lbls_test1, probs_test2, lbls_test2 = \
                x_epitope_run(ag_train, lbls_train, masks_train, lengths_train,
                              cdrs_train, cdr_masks_train, cdr_lengths_train,
                              weights_template, i,
                              ag_test, lbls_test, masks_test, lengths_test,
                              cdrs_test, cdr_masks_test, cdr_lengths_test)


        lbls_test2 = np.squeeze(lbls_test2)
        all_lbls2 = np.concatenate((all_lbls2, lbls_test2))
        all_lbls1.append(lbls_test1)

        probs_test_pad = torch.zeros(probs_test1.data.shape[0], MAX_EXT_AG_LENGTH, probs_test1.data.shape[2])
        probs_test_pad[:probs_test1.data.shape[0], :probs_test1.data.shape[1], :] = probs_test1.data
        probs_test_pad = Variable(probs_test_pad)

        probs_test2 = np.squeeze(probs_test2)
        #print(probs_test)
        all_probs2 = np.concatenate((all_probs2, probs_test2))
        #print(all_probs)
        #print(type(all_probs))

        all_probs1.append(probs_test_pad)

        all_masks.append(masks_test)

    lbl_mat1 = torch.cat(all_lbls1)
    prob_mat1 = torch.cat(all_probs1)
    #print("end", all_probs)
    mask_mat = torch.cat(all_masks)

    with open(output_file, "wb") as f:
        pickle.dump((lbl_mat1, prob_mat1, mask_mat, all_lbls2, all_probs2), f)

Extracting epitopes from all the pdbs


In [128]:
summary_files = pd.read_pickle("/content/drive/My Drive/Peritia_Fast-Parapred/summary_file_cleaned.pkl")
summary_files = summary_files.drop(summary_files[summary_files.pdb.isin(["4ydl","4ydv","5ig7","5ies","4hii","5w0d","5i9q","4hkx",
                                                          "5te4","5te6","4xvs","4xvt","3gbm","2xqb","5if0","4s1q",
                                                          "4xmp","4xny","3mlt","6mvl","4xh2","5ifj","5te7","5occ",
                                                          "4dqo","4lsq","4lsp","4lsr","4yaq","3h3p","3idi",
                                                          "3lrs","3qnz","5x08","5alc","2p8m","6pbw","5csz",
                                                          "6bkb","5ado","4xaw","5wnb","5dt1","4ob5","4xbe",
                                                          "5fgb","4y5y","3u4e","4xcf","4rnr","3drq","6uyf",
                                                          "4m8q","3d0l","2jb6","3u2s","3lhp","3mug","1bvk","5e08","5gkr",
                                                                        "6mwn","6u8k","6u8d","6df1"])].index)

pdb_names = summary_files['pdb'].to_numpy()
Hchain_id = summary_files['Hchain'].to_numpy()
Lchain_id = summary_files['Lchain'].to_numpy()
Achain_id = summary_files['antigen_chain_list'].to_numpy()
 
epi_res = []
epi_res_num = []
ext_epi_res_num = []
ext_epi_res = []
A_pdb = []
# for i in range(len(pdb_names)):
#     epi_residues,epi_res_number,ext_epi_res_number, ext_epi_residues = ext_epitope_sorted(Hchain_id[i],Lchain_id[i],Achain_id[i],pdb_names[i])
#     A_pdb.append(pdb_names[i])
#     epi_res.append(epi_residues)
#     epi_res_num.append(epi_res_number)
#     ext_epi_res_num.append(ext_epi_res_number)
#     ext_epi_res.append(ext_epi_residues)

In [129]:
# Sorted_Antigen_dataframe = pd.DataFrame(list(zip(A_pdb,epi_res,epi_res_num, ext_epi_res_num, ext_epi_res)),
#                columns =['pdb','sorted_epi_residues','sorted_epi_res_number', 'sorted_ext_epi_res_number', 'sorted_ext_epi_residues'])
# Sorted_Antigen_dataframe.to_pickle("/content/drive/My Drive/Peritia_Fast-Parapred/Sorted_Antigen_epitope_dataframe.pkl")

In [130]:
 Sorted_epi = pd.read_pickle("/content/drive/My Drive/Peritia_Fast-Parapred/Sorted_Antigen_epitope_dataframe.pkl")
 Sorted_epi.head()

,pdb,sorted_epi_residues,sorted_epi_res_number,sorted_ext_epi_res_number,sorted_ext_epi_residues
0,4ydk,"[Q, L, T, G, G, S, T, E, N, A, K, T, Q, P, S, ...","[105 , 122 , 123 , 124 , 198 , 199 , 257 , 275...","[52 , 54 , 65 , 66 , 69 , 93 , 94 , 95 , 96 , ...","[L, C, V, H, W, F, N, M, W, K, N, N, M, V, E, ..."
1,4ydj,"[W, K, E, E, N, N, A, K, T, I, R, P, S, G, G, ...","[96 , 97 , 102 , 275 , 279 , 280 , 281 , 282 ,...","[47 , 48 , 49 , 50 , 51 , 52 , 69 , 92 , 93 , ...","[D, A, D, T, T, L, W, N, F, N, M, W, K, N, N, ..."
2,1u8i,"[E, L, D, K, W, A, N]","[1 , 2 , 3 , 4 , 5 , 6 , 7 ]","[1 , 2 , 3 , 4 , 5 , 6 , 7 ]","[E, L, D, K, W, A, N]"
3,1u8l,"[D, L, D, R, W, A, S]","[1 , 2 , 3 , 4 , 5 , 6 , 7 ]","[1 , 2 , 3 , 4 , 5 , 6 , 7 ]","[D, L, D, R, W, A, S]"
4,5y9j,"[K, G, S, Y, T, Y, A, M, G, H, L, Q, K, V, G, ...","[160 , 161 , 162 , 163 , 205 , 206 , 207 , 208...","[156 , 157 , 158 , 159 , 160 , 161 , 162 , 163...","[P, T, I, Q, K, G, S, Y, T, F, V, W, E, N, K, ..."


In [131]:
epi_res = Sorted_epi['sorted_epi_residues'].to_numpy()
epi_res_num = Sorted_epi['sorted_epi_res_number'].to_numpy()
ext_epi_res_num = Sorted_epi['sorted_ext_epi_res_number'].to_numpy()
ext_epi_res = Sorted_epi['sorted_ext_epi_residues'].to_numpy()

**Building input data Matrix**

In [132]:
##Joining all the residue abbrevations for making a sequence and repeating each seq 6 times/pdb

Anti_str = []
for i in range(len(ext_epi_res)):
#     print(i)
    str1 = ''.join(ext_epi_res[i])
#     print(repeat(str1, 6))
#     print(str1,i,len(A_mat[i]))
    Anti_str.append(str1)

import itertools
Anti_seq = list(itertools.chain.from_iterable(itertools.repeat(x, 6) for x in Anti_str))

MAX_EXT_AG_LENGTH = maxi_len(Anti_seq)
MAX_EXT_AG_LENGTH

288

In [133]:
## Generating Labels for Extended Epitope Antigen Matrix

labels = []
for i in range(len(ext_epi_res_num)):
    label = []
    for j in ext_epi_res_num[i]:
#     print(i)
        if j in epi_res_num[i]:
            label.append(1)
        else:
            label.append(0)
    labels.append(label)

import itertools
Anti_repeat_labels = list(itertools.chain.from_iterable(itertools.repeat(x, 6) for x in labels))

# Antigen_lbls = np.stack(labels)
MAX_EXT_AG_LENGTH = 288

label_mats = []
for i in range(len(Anti_repeat_labels)):
    label_mat = torch.tensor(Anti_repeat_labels[i])
    label_mat_pad = torch.zeros((MAX_EXT_AG_LENGTH, 1))
    label_mat_pad[:label_mat.shape[0], 0] = label_mat
    label_mats.append(label_mat_pad)

ext_epi_lbls = torch.stack(label_mats)
ext_epi_lbls.shape

torch.Size([3774, 288, 1])

In [134]:
NUM_FEATURES = 28
MAX_EXT_AG_LENGTH = 288
# cdr_mats = []
ext_epi_mats = []

ext_epi_lengths = []
for i in range(len(Anti_seq)):
#     on_hot = seq_to_one_hot(df['CDR'][i])
#     print(i)
    ext_epi_mat = seq_to_one_hot(Anti_seq[i])
    ext_epi_mat_pad = torch.zeros((MAX_EXT_AG_LENGTH, NUM_FEATURES))
    ext_epi_mat_pad[:ext_epi_mat.shape[0], :] = ext_epi_mat
    ext_epi_mats.append(ext_epi_mat_pad)
    ext_epi_lengths.append(ext_epi_mat.shape[0])

ext_epi = torch.stack(ext_epi_mats)
ext_epi.shape


torch.Size([3774, 288, 28])

In [135]:
max(ext_epi_lengths)

288

In [136]:
ext_epi_masks = []
for i in range(len(Anti_seq)):
    ext_epi_mask = torch.zeros((MAX_EXT_AG_LENGTH, 1), dtype=int)
    ext_epi_mask[:len(Anti_seq[i]), 0] = 1
    ext_epi_masks.append(ext_epi_mask)
ext_epi_masks = torch.stack(ext_epi_masks)
ext_epi_masks.shape

torch.Size([3774, 288, 1])

In [137]:
NUM_FEATURES = 28+6
Max_len_CDR = 38
cdr_mats = []

cdr_loop = pd.read_pickle("/content/drive/My Drive/Peritia_Fast-Parapred/CDRs_seqno_pdb.pkl")
cdr_loop = cdr_loop.drop(cdr_loop[cdr_loop.PDB_Name.isin(["4ydl","4ydv","5ig7","5ies","4hii","5w0d","5i9q","4hkx",
                                                          "5te4","5te6","4xvs","4xvt","3gbm","2xqb","5if0","4s1q",
                                                          "4xmp","4xny","3mlt","6mvl","4xh2","5ifj","5te7","5occ",
                                                          "4dqo","4lsq","4lsp","4lsr","4yaq","3h3p","3idi",
                                                          "3lrs","3qnz","5x08","5alc","2p8m","6pbw","5csz",
                                                          "6bkb","5ado","4xaw","5wnb","5dt1","4ob5","4xbe",
                                                          "5fgb","4y5y","3u4e","4xcf","4rnr","3drq","6uyf",
                                                          "4m8q","3d0l","2jb6","3u2s","3lhp","3mug","1bvk","5e08","5gkr",
                                                                        "6mwn","6u8k","6u8d","6df1"])].index)
cdr_loop.reset_index(drop=True, inplace=True)

for i in range(len(cdr_loop['CDR'])):
#     on_hot = seq_to_one_hot(df['CDR'][i])
    cdr_mat = cdrseq_to_one_hot(cdr_loop['CDR'][i],i)
    cdr_mat_pad = torch.zeros((Max_len_CDR, NUM_FEATURES))
    cdr_mat_pad[:cdr_mat.shape[0], :] = cdr_mat
    cdr_mats.append(cdr_mat_pad)
cdrs_seq = torch.stack(cdr_mats)
cdrs_seq.shape

torch.Size([3774, 38, 34])

In [138]:
all_cdrs_lengths = []
for i in range(len(cdr_loop['CDR'])):
#     on_hot = seq_to_one_hot(df['CDR'][i])
    cdr_mat = cdrseq_to_one_hot(cdr_loop['CDR'][i],i)
    cdr_mat_pad = torch.zeros((Max_len_CDR, NUM_FEATURES))
    cdr_mat_pad[:cdr_mat.shape[0], :] = cdr_mat
    cdr_mats.append(cdr_mat_pad)
    all_cdrs_lengths.append(cdr_mat.shape[0])
max(all_cdrs_lengths)

28

In [139]:
Max_len_CDR = 38

cdr_masks = []
for i in range(len(cdr_loop['CDR'])):
    cdr_mask = torch.zeros((Max_len_CDR, 1), dtype=int)
    cdr_mask[:len(cdr_loop['CDR'][i]), 0] = 1
    cdr_masks.append(cdr_mask)
cdr_mask = torch.stack(cdr_masks)
cdr_mask.shape

torch.Size([3774, 38, 1])

In [141]:
cdr_loop = pd.read_pickle("/content/drive/My Drive/Peritia_Fast-Parapred/CDRs_seqno_pdb.pkl")
cdr_loop = cdr_loop.drop(cdr_loop[cdr_loop.PDB_Name.isin(["4ydl","4ydv","5ig7","5ies","4hii","5w0d","5i9q","4hkx",
                                                          "5te4","5te6","4xvs","4xvt","3gbm","2xqb","5if0","4s1q",
                                                          "4xmp","4xny","3mlt","6mvl","4xh2","5ifj","5te7","5occ",
                                                          "4dqo","4lsq","4lsp","4lsr","4yaq","3h3p","3idi",
                                                          "3lrs","3qnz","5x08","5alc","2p8m","6pbw","5csz",
                                                          "6bkb","5ado","4xaw","5wnb","5dt1","4ob5","4xbe",
                                                          "5fgb","4y5y","3u4e","4xcf","4rnr","3drq","6uyf",
                                                          "4m8q","3d0l","2jb6","3u2s","3lhp","3mug","1bvk",
                                                          "5e08","5gkr","6mwn","6u8k","6u8d","6df1"])].index)
cdr_loop.reset_index(drop=True, inplace=True)

dist_matx_path_light = "/content/drive/My Drive/Peritia_Fast-Parapred/Light_chain_dist_clean/"
dist_matx_path_heavy = "/content/drive/My Drive/Peritia_Fast-Parapred/Heavy_chain_dist_clean/"
cut_off = 5
# all_variables = get_variables(dist_matx_path_heavy,dist_matx_path_light,cdr_loop,cut_off)
with open('/content/drive/My Drive/Peritia_Fast-Parapred/all_variables.pkl', 'rb') as f:
    all_variables = pickle.load(f)
# all_variables


In [142]:
cont_mats = []
cdr_lengths = []
Max_len_CDR = 38
"""
Pad the variables to Max CDR len with 0
Here max len of CDR is 38

E.g - Input = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
    Output = [0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[1.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.]

"""
for i in range(len(all_variables)):
    cont_mat = torch.tensor(all_variables[i])
    cont_mat_pad = torch.zeros((Max_len_CDR, 1))
    cont_mat_pad[:cont_mat.shape[0], 0] = cont_mat
    cont_mats.append(cont_mat_pad)
    cdr_lengths.append(cont_mat.shape[0])


cdr_lbls = torch.stack(cont_mats)

cdr_lbls.shape
# cdr_lengths

torch.Size([3774, 38, 1])

In [143]:
def structure_ids_to_selection_mask(idx, num_structures):
    mask = np.zeros((num_structures * 6, ), dtype=np.bool)
    offset = idx * 6
    for i in range(6):
        mask[offset + i] = True
    return mask


In [144]:
MAX_CDR_LENGTH = 38

AG_NUM_FEATURES = 28

NUM_FEATURES = 34

MAX_EXT_AG_LENGTH = 288

use_cuda = torch.cuda.is_available()

epochs = 16

batch_size = 3

In [145]:
NUM_ITERATIONS = 10
NUM_SPLIT = 10
output_file="crossval-data.p",
weights_template="weights-fold-{}.h5"
import os 
def run_cv(output_folder="cv-ag-seq",
           num_iters=NUM_ITERATIONS):
#     cache_file = dataset.split("/")[-1] + ".p"
#     dataset = open_dataset(dataset_cache=cache_file)

    dir =  output_folder + "/weights"
    if not os.path.exists(dir):
        os.makedirs(output_folder + "/weights")
    for i in range(num_iters):
        #i=0
        print("Crossvalidation run", i+1)
        output_file = "{}/run-{}.p".format(output_folder, i)
        weights_template = output_folder + "/weights/run-" + \
                           str(i) + "-fold-{}.pth.tar"
        kfold_cv_eval(ext_epi, ext_epi_lbls, ext_epi_masks, ext_epi_lengths, cdrs_seq,cdr_mask,all_cdrs_lengths ,
                      i,output_file, weights_template)


In [147]:
import time
import os
start_time = time.time()
run_cv(output_folder="/content/drive/My Drive/Peritia_Fast-Parapred/cv-ag-seq",
           num_iters=NUM_ITERATIONS)
end_time = time.time() - start_time
hours, rem = divmod(end_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

Crossvalidation run 1
ag torch.Size([3774, 288, 28])
lbls torch.Size([3774, 288, 1])
masks torch.Size([3774, 288, 1])
cdrs torch.Size([3774, 38, 34])
cdr masks torch.Size([3774, 38, 1])
train_idx, test_idx: (array([   0,    3,    6, ..., 3770, 3771, 3772]), array([   1,    2,    4, ..., 3765, 3769, 3773]))
Fold:  1
len(train_idx) 2516
epitope run
using cuda


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 0 - loss is 1.204668 : 
Roc 0.6380423018721144
Epoch 1 - loss is 0.991659 : 
Roc 0.6630710678319852
Epoch 2 - loss is 0.972102 : 
Roc 0.6442031573740912
Epoch 3 - loss is 0.957234 : 
Roc 0.649968109336591
Epoch 4 - loss is 0.938730 : 
Roc 0.7237862208474369
Epoch 5 - loss is 0.923659 : 
Roc 0.71290867010326
Epoch 6 - loss is 0.907598 : 
Roc 0.7273007260420783
Epoch 7 - loss is 0.892767 : 
Roc 0.7919702150453798
Epoch 8 - loss is 0.871271 : 
Roc 0.6835464565547641
Epoch 9 - loss is 0.839724 : 
Roc 0.7949645279431171
Epoch 10 - loss is 0.819758 : 
Roc 0.8032576857464235
Epoch 11 - loss is 0.805997 : 
Roc 0.792213119160152
Epoch 12 - loss is 0.803086 : 
Roc 0.7781887932613049
Epoch 13 - loss is 0.793016 : 
Roc 0.8232585363346732


KeyboardInterrupt: ignored

In [149]:
ext_epi[0]
Anti_seq[0]

'LCVHWFNMWKNNMVEQMQEDVISLWDQSLQPCVKLTGGSVIKQFPIYFNGTGIKPVVSTQLLLGIIRSENLTNNAKTIIVHICRACWLVTLKHFNNKTIIFQPPSGGDLEITMHHFNCGEFFYCNTTQLFNNTILPCKIKQIINMWQGTGQAMYSITGILLTRDGGANTSNETFRPGGGNIKDNWRSELY'

In [150]:
model = EpitopeX()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [152]:
prediction = model([ext_epi[0]])

TypeError: ignored